In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
import glob
import os
fn = glob.glob("./drive/My Drive/Spliter/class_training/*")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
fn

In [0]:
import tensorflow as tf
from keras import backend as K
from keras.layers import *
from keras.models import Model
import sys

py_version = '.'.join([str(i) for i in list(sys.version_info[:3])])
tf_version = int(''.join(str(tf.__version__).split('.')[:2]))
_log = __import__('tensorflow.compat.v1', fromlist=['log'], level=0) if tf_version > 15 else __import__('tensorflow',
                                                                                                        fromlist=[
                                                                                                            'log'],
                                                                                                        level=0)

BN = BatchNormalization(beta_initializer=tf.zeros_initializer(),
                                    gamma_initializer=tf.ones_initializer()
                                    , moving_mean_initializer=tf.zeros_initializer(),
                                    moving_variance_initializer=tf.ones_initializer())


class Self_Attention(Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Self_Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(3, input_shape[2], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)

        super(Self_Attention, self).build(input_shape)

    def call(self, x):
        WQ = K.dot(x, self.kernel[0])
        WK = K.dot(x, self.kernel[1])
        WV = K.dot(x, self.kernel[2])

        QK = K.batch_dot(WQ, K.permute_dimensions(WK, [0, 2, 1]))
        QK = QK / (64 ** 0.5)
        QK = K.softmax(QK)

        V = K.batch_dot(QK, WV)
        return V

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.output_dim)


def res_conv2D_block(input_: Model or Layer, filters: int, kernel_size: tuple or list, padding: str = 'SAME',
                     strides: tuple or list = (1, 1), D_forge: int = 2, repeat_T: int = 1, name_scope: int = 0,
                     activation: str or Layer = LeakyReLU(0.3)):
    for it in range(repeat_T):
        model = input_
        for i in range(D_forge):
            model = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                           padding=padding, name="res_Conv2D_{}_{}_{}".format(name_scope, it, i))(model)
        model = activation(model)
        input_ = Add()([model, input_])
        input_ = BatchNormalization(beta_initializer=tf.zeros_initializer(),
                                    gamma_initializer=tf.ones_initializer()
                                    , moving_mean_initializer=tf.zeros_initializer(),
                                    moving_variance_initializer=tf.ones_initializer())(input_)
    return input_


def res_Dense_block(input_: Model or Layer, D_forge: int = 2, repeat_T: int = 1, name_scope: int = 0,
                    activation: str or Layer = LeakyReLU(0.3), target_block: object or Layer = Dense):
    target_block = target_block if type(target_block) is Layer else Dense
    for it in range(repeat_T):
        model = input_
        for i in range(D_forge):
            model = target_block(int(model.get_shape()[-1]), activation='linear',
                                 name="res_Dense_{}_{}_{}".format(name_scope, it, i))(model)
        model = activation(model)
        input_ = Add()([model, input_])
        input_ = BatchNormalization(momentum=0.9)(input_)
        input_ = Dropout(0.25)(input_)
    return input_


class Self_define_loss(object):
    def __init__(self, cls_num:int=7):
        self.cls_num = cls_num

    def mycrossentropy(self, y_true, y_pred, e=0.1):
        loss1 = K.categorical_crossentropy(y_true, y_pred)
        loss2 = K.categorical_crossentropy(K.ones_like(y_pred)/self.cls_num, y_pred)
        return (1-e)*loss1 + e*loss2

    def multi_category_focal_loss2_fixed(self, y_true, y_pred):
        epsilon = 1.e-7
        gamma = 2.
        alpha = tf.constant(0.5, dtype=tf.float32)

        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        alpha_t = y_true * alpha + (tf.ones_like(y_true) - y_true) * (1 - alpha)
        y_t = tf.multiply(y_true, y_pred) + tf.multiply(1 - y_true, 1 - y_pred)
        ce = log(y_t)
        weight = tf.pow(tf.subtract(1., y_t), gamma)
        fl = tf.multiply(tf.multiply(weight, ce), alpha_t)
        loss = tf.reduce_mean(fl)
        return loss


    def root_mean_squared_error(self, y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))


def res_Self_attention_block(input_: Model or Layer, D_forge: int = 2, repeat_T: int = 1, name_scope: int = 0):
    for it in range(repeat_T):
        model = input_
        for i in range(D_forge):
            model = Self_Attention(int(model.get_shape()[-1]), name="res_Self_attention_{}_{}_{}".format(name_scope, it, i))(model)
        input_ = Add()([model, input_])
        input_ = BatchNormalization(momentum=0.9)(input_)
        input_ = Dropout(0.25)(input_)
    return input_

Using TensorFlow backend.


In [0]:
import glob
import os
import sys
import tensorflow as tf
import numpy as np
import pandas as pd

from keras import backend as K
from keras.callbacks import ReduceLROnPlateau
from keras.layers import *
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.utils import to_categorical, plot_model

configs = {
    'fit_generator': False,
    'allow_growth': True,
    'batch_size': 50,
}

py_version = '.'.join([str(i) for i in list(sys.version_info[:3])])
tf_version = int(''.join(str(tf.__version__).split('.')[:2]))

_log = __import__('tensorflow.compat.v1', fromlist=['log'], level=0) if tf_version > 15 else __import__('tensorflow',
                                                                                                        fromlist=[
                                                                                                            'log'],
                                                                                                        level=0)
log = _log.log

fn = glob.glob("./Drive/Spliter/*")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
label_name = []


def detect_data_path(fn: glob.glob or None) -> dict:
    result = {}
    folder_path = [i.replace('\\', '/') if " " in str(i).split('\\')[-1] else None for i in fn]
    if all([True if i is None else False for i in folder_path]):
        folder_path = [i.replace('\\', '/') for i in fn]
        print(folder_path)
    else:
        [folder_path.remove(None) if i is None else i for i in folder_path.copy()]
    file_name = [str(i).split('/')[-1].split('\\')[-1] if '\\' in str(i).split('/')[-1] else str(i).split('/')[-1] for i
                 in folder_path]

    for it, main_key in enumerate(file_name):
        data_file_path = [i.replace('\\', '/') for i in glob.glob(f"{folder_path[it]}/*")]
        data_file_name = [str(i).split('/')[-1] for i in data_file_path]
        temp_dict = {data_file_name[i]: m for i, m in enumerate(data_file_path)}
        temp_dict.update({'stock_id': str(folder_path[it]).split('/')[-1].split(' ')[0]})
        result.update({main_key: temp_dict})

    return result


def read_data(stock: str, use_set_remove: bool = True,
              data_main_floder: str or glob.glob or None = None) -> pd.DataFrame:
    data_dict = detect_data_path(data_main_floder)
    df = pd.DataFrame(data_dict)
    x = {df[i]['stock_id']: i for i in df} if stock.isdigit() else None
    stock = x[stock] if x is not None and stock in x else stock
    x_df, y_df = [[pd.read_csv(df[i]['Tech_I.csv'], header=0) for i in list(df.keys())],
                  [pd.read_csv(df[i]['y_train_change_class.csv'], header=0) for i in
                   list(df.keys())]] if stock in [
        "all", "ALL"] else [pd.read_csv(df[stock]['Tech_I.csv'], header=0),
                            pd.read_csv(df[stock]['y_train_change_class.csv'], header=0)]

    x_df, y_df = [pd.concat(x_df, axis=0), pd.concat(y_df, axis=0)] if stock in ["all", "ALL"] else (x_df, y_df)

    if use_set_remove:
        x_df_c, y_df_c = list(x_df.columns), list(y_df.columns)
        same_col = set(x_df_c)
        same_col.intersection_update(y_df_c)
    else:
        same_col = []
    if len(same_col) is not 0: y_df.drop(str(same_col.pop()), axis=1)
    result = pd.concat([x_df, y_df], axis=1, join='inner')
    return result


def train_test(data: pd.DataFrame, batch_size, timestep, feature_dim, FD, drop_list: list or None = None,
               label_name: list or None = None, use_gaussian: bool = True, label_num: list = [], to_cat: bool = True,
               preds: bool = False):
    def de_shape(feature: np.array, batch_size=None, timestep=None, feature_dim=None, FD=None):
        shape_data = [-1 if m is None and i is 0 else m for i, m in enumerate([batch_size, timestep, feature_dim, FD])]
        [shape_data.remove(None) if i is None else i for i in shape_data.copy()]
        feature = feature.reshape(shape_data)
        return feature

    def drop_dim(data: pd.DataFrame, drop_list):
        data = data.drop(drop_list, axis=1) if isinstance(drop_list, list) else data.drop(["date"], axis=1)
        return data

    def find_labels():
        res = -1 if label_name is None else [list(data.keys()).index(i) for i in label_name]
        res.sort() if isinstance(res, list) else None
        return res[0] if isinstance(res, list) else res

    def gaussian_labels(D: pd.DataFrame, label_num: list, stage: list or None = None, deg: float = 1.0):
        stage = [0.63, 1] if stage is None else stage
        result = []

        for i, m in enumerate(label_num):
            label = list(to_categorical(pd.DataFrame(D)[i], num_classes=m)) if to_cat is True else list(
                pd.DataFrame(D)[i])

            if use_gaussian:
                res = []
                for ic in label:
                    item = list(ic)
                    _index = item.index(1)
                    item[_index] = deg
                    for it in range(1, stage[1] + 1):
                        if _index - it >= 0:
                            item[_index - it] = stage[0] ** (it)
                        if _index + it < m:
                            item[_index + it] = stage[0] ** (it)
                    res.append(item)
                result.append(list(de_shape(np.array(res), batch_size, m)))
            else:
                result.append(list(label))

        return result

    def train_test(data, row: float or None = None):
        row = round(0.85 * data.shape[0]) if row is None else row
        if preds:
            return np.array(data[:int(row)]), np.array(data[-1:])
        else:
            return np.array(data[:int(row)]), np.array(data[int(row):])

    def normal_categorical(D: pd.DataFrame, label_num: list or None = None) -> list:
        label_len = np.array(D).shape[-1]
        if label_num is None: label_num = []
        result = []
        for i in range(len(label_num), label_len):
            if to_cat:
                max_label_num = int(max(pd.DataFrame(D)[i]) + 1)
                result.append(list(to_categorical(pd.DataFrame(D)[i], num_classes=max_label_num)))
            else:
                result.append(list(pd.DataFrame(D)[i]))
        return result

    data = drop_dim(data, drop_list)
    labels_cl = find_labels()

    train, test = train_test(data)

    x_train = train[:, :labels_cl]
    label_train = train[:, labels_cl:]

    x_test = test[:, :labels_cl]
    label_test = test[:, labels_cl:]

    feature_train = de_shape(x_train, batch_size, timestep, feature_dim, FD)
    feature_test = de_shape(x_test, batch_size, timestep, feature_dim, FD)
    if use_gaussian:
        # label_num= [label_num, label_num, label_num=39]
        label_train_ = gaussian_labels(label_train, label_num=label_num)
        label_test_ = gaussian_labels(label_test, label_num=label_num)
        if to_cat:
            label_test_.append(list(to_categorical(pd.DataFrame(label_test)[3], num_classes=3)))
            label_train_.append(list(to_categorical(pd.DataFrame(label_train)[3], num_classes=3)))
        else:
            label_test_.append(list(pd.DataFrame(label_test)[3]))
            label_train_.append(list(pd.DataFrame(label_train)[3]))
    else:
        label_train_ = normal_categorical(label_train, label_num)
        label_test_ = normal_categorical(label_test, label_num)

    return feature_train, [np.array(i) for i in label_train_], feature_test, [np.array(i) for i in label_test_]


def data_l(stock_id, target, csv_floder_path=None, preds: bool = False, label_name: list = []):
    result = read_data(stock_id, data_main_floder=csv_floder_path)
    drop_list = ["date"]
    re_dict = {
            'high': '0',
            'low': '1',
            'close': '2',
            'trend': '3'
        }
    # re_dict = {m: i for i, m in enumerate(label_name)}
    feature_train, train_labels, feature_test, test_labels = train_test(result, -1, None, 900, None,
                                                                        drop_list,
                                                                        label_name=label_name,
                                                                        use_gaussian=False,
                                                                        label_num=[],
                                                                        to_cat=True, preds=preds)
    ret = {str(i): [m, test_labels[i]] for i, m in enumerate(train_labels)}
    result = ret[re_dict[target]]
    return feature_train, result[0], feature_test, result[1]


def odd_data_l(stock_id, target):
    dic = fn
    result = read_data(stock_id, data_main_floder=dic)
    drop_list = ["date"]
    label_name = ['high_change_class', 'low_change_class', 'close_change_class', 'close_change_class3']
    feature_train, [high_label_train, low_label_train, close_label_train, trend_label_train], feature_test, [
        high_label_test, low_label_test, close_label_test, trend_label_test] = train_test(result, -1, None, 900, None,
                                                                                          drop_list,
                                                                                          label_name,
                                                                                          use_gaussian=False,
                                                                                          label_num=[],
                                                                                          to_cat=True)
    ret = {
        'ALL': [[high_label_train, low_label_train, close_label_train, trend_label_train],
                [high_label_test, low_label_test, close_label_test, trend_label_test]],
        'high': [high_label_train, high_label_test],
        'low': [low_label_train, low_label_test],
        'close': [close_label_train, close_label_test],
        'trend': [trend_label_train, trend_label_test],
    }

    result = ret[target]

    return feature_train, result[0], feature_test, result[1]


def auto_multilabels_to_one(em_len, features: list, labels: list, output_dim: int, define_loss='mean_squared_error'):
    split_time = round(int(np.array(features[0]).shape[-1]) / em_len)

    Feature_train_ = [features[0][:, i * split_time:i * split_time + split_time] for i in range(em_len)]
    Feature_test_ = [features[1][:, i * split_time:i * split_time + split_time] for i in range(em_len)]

    Label_train = {'out_{}'.format(i): np.array(m).reshape([-1, split_time, 1]) for i, m in enumerate(Feature_train_)}
    Feature_train = {'intput_{}'.format(i): np.array(m).reshape([-1, split_time, 1]) for i, m in
                     enumerate(Feature_train_)}

    Label_test = {'out_{}'.format(i): np.array(m).reshape([-1, split_time, 1]) for i, m in enumerate(Feature_test_)}
    Feature_test = {'intput_{}'.format(i): np.array(m).reshape([-1, split_time, 1]) for i, m in
                    enumerate(Feature_test_)}

    Loss = {'out_{}'.format(i): define_loss for i in range(em_len)}
    Loss_weights = {'out_{}'.format(i): 1. for i in range(em_len)}
    # --------------------------------------------------------------------------------------------------------------#
    ouput_name = 'final_output'
    Label_train.update({ouput_name: np.array(labels[0].reshape([-1, output_dim]))})
    Label_test.update({ouput_name: np.array(labels[1].reshape([-1, output_dim]))})
    Loss.update({ouput_name: 'categorical_crossentropy'})
    Loss_weights.update({ouput_name: 1.})

    # --------------------------------------------------------------------------------------------------------------#
    return Feature_train, Label_train, Feature_test, Label_test, Loss, Loss_weights

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

def train_mode(target: str, stock_id, model, output_dim, define_loss, input_num, label_name, data_path):
    feature_train, label_train, feature_test, label_test = data_l(stock_id, target,label_name=label_name,preds=False,
                                                                  csv_floder_path=data_path)
    FS = [feature_train, feature_test]
    LS = [label_train, label_test]
    Feature_train, Label_train, Feature_test, Label_test, Loss, Loss_weights = auto_multilabels_to_one(input_num, FS,
                                                                                                       LS, output_dim,
                                                                                                       define_loss=define_loss)
    opt = Adam(1e-5)
    model.compile(optimizer=opt, loss=Loss, loss_weights=Loss_weights, metrics=['acc'])
    model.summary()
    return model, [Feature_train, Label_train, Feature_test, Label_test], [feature_train, label_train, feature_test,
                                                                           label_test]


def fit_model(epochs: int = 10, batch_size: int = 50, define_loss='categorical_crossentropy',
              reduce_lr: bool = True,
              is_train: bool = True, input_num=0, data_path='',label_name=[],save_path=''):
    callbacks = []
    result = train_mode(target='close', stock_id='2330 台積電', model=model_train, output_dim=output_dim,
                        define_loss=define_loss, input_num=input_num,label_name=label_name,data_path=data_path)

    if is_train:
        callbacks.append(
            ModelCheckpoint(f'.{save_path}.'+'{epoch:05d}-{val_loss:.5f}.hdf5', save_best_only=True))

    if reduce_lr:
        callbacks.append(ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=50, min_lr=0.00000001, verbose=1))

    result[0].fit(result[1][0], result[1][1], epochs=epochs, batch_size=batch_size, callbacks=callbacks,
                  validation_split=0.01, verbose=1)
    return result

In [0]:
def layers(input_target):
    conv1D_list = [Conv1D(32, 1, name='Conv1D_0_{}'.format(i))(m) for i, m in enumerate(input_target)]
    res_dense = [res_Dense_block(m, name_scope='0_{}'.format(i)) for i, m in enumerate(conv1D_list)]
    BN = [BatchNormalization()(m) for i, m in enumerate(res_dense)]
    res_dense = [res_Dense_block(m, name_scope='1_{}'.format(i)) for i, m in enumerate(BN)]
    mid_layer = [TimeDistributed(Dense(128, activation='linear', name='mid_Dense_{}'.format(i)))(m) for i, m in
                 enumerate(res_dense)]
    res_dense = [res_Dense_block(m,name_scope='2_{}'.format(i)) for i, m in enumerate(mid_layer)]
    BN = [BatchNormalization()(m) for i, m in enumerate(res_dense)]
    res_dense = [res_Dense_block(m,name_scope='3_{}'.format(i)) for i, m in enumerate(BN)]
    conv1D_list = [Conv1D(32, 1, name='Conv1D_1_{}'.format(i))(m) for i, m in enumerate(res_dense)]

    auto_encoder_out = [TimeDistributed(Dense(1, activation='linear'), name='out_{}'.format(i))(m) for i, m in
                        enumerate(conv1D_list)]
    return auto_encoder_out, mid_layer


def output_layer(input_target, output_dim):
    model = Concatenate()(input_target)
    model = BatchNormalization()(model)
    model = res_Dense_block(model, name_scope=2)
    model = res_Dense_block(model, name_scope=3)
    model = BatchNormalization()(model)
    model = LeakyReLU()(model)
    model = Self_Attention(90, name="Self_0")(model)
    model = BatchNormalization()(model)
    model = Self_Attention(90, name="Self_1")(model)
    model = BatchNormalization()(model)
    avg_model = GlobalAveragePooling1D()(model)
    max_model = GlobalMaxPooling1D()(model)
    model = Concatenate()([avg_model, max_model])
    model = BatchNormalization()(model)
    model = LeakyReLU()(model)
    model = Dense(output_dim, activation='linear', name='final_output')(model)

    return model

In [0]:
#config
output_dim = 7
emb_max_input = 700
input_dim = 90
batch_size = 50
epochs = 30000
input_num = 10
data_path = glob.glob("./drive/My Drive/Spliter/class_training/*")
label_name = ['high_change_class', 'low_change_class', 'close_change_class', 'close_change_class3']
define_loss = Self_define_loss().root_mean_squared_error

In [0]:
input_target = [Input(shape=(input_dim, 1), name='intput_{}'.format(i)) for i in range(10)]
model = layers(input_target)
model[0].append(output_layer(model[1], output_dim))
save_path = '/drive/My Drive/Model_save/weights'
model_train = Model(inputs=input_target, outputs=model[0])

In [0]:
result = fit_model(epochs=epochs, batch_size=batch_size, define_loss=define_loss,
                   input_num=input_num, data_path=data_path, label_name=label_name,
                   save_path=save_path)

2083/2083 [==============================] - 9s 4ms/step - loss: 17.8640 - out_0_loss: 1.1684 - out_1_loss: 1.2592 - out_2_loss: 1.2922 - out_3_loss: 1.1954 - out_4_loss: 1.0968 - out_5_loss: 1.0878 - out_6_loss: 1.2050 - out_7_loss: 0.8090 - out_8_loss: 1.3972 - out_9_loss: 1.1499 - final_output_loss: 6.2033 - out_0_acc: 2.6671e-05 - out_1_acc: 4.8008e-05 - out_2_acc: 5.8676e-05 - out_3_acc: 4.8008e-05 - out_4_acc: 3.7339e-05 - out_5_acc: 3.7339e-05 - out_6_acc: 3.7339e-05 - out_7_acc: 4.8008e-05 - out_8_acc: 1.6003e-05 - out_9_acc: 3.7339e-05 - final_output_acc: 0.1099 - val_loss: 9.9148 - val_out_0_loss: 0.1350 - val_out_1_loss: 0.1236 - val_out_2_loss: 0.2991 - val_out_3_loss: 0.0993 - val_out_4_loss: 0.2914 - val_out_5_loss: 0.2813 - val_out_6_loss: 0.1548 - val_out_7_loss: 0.2036 - val_out_8_loss: 0.0655 - val_out_9_loss: 0.1327 - val_final_output_loss: 8.1285 - val_out_0_acc: 0.0000e+00 - val_out_1_acc: 0.0000e+00 - val_out_2_acc: 0.0000e+00 - val_out_3_acc: 0.0000e+00 - val_out

In [0]:
plot_model(model_train,show_shapes=True)

In [0]:
#打開Ngrolk並綁定port:6006
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);